In [1]:
import tensorflow as tf
from tensorflow import keras
import datasets
import os

In [ ]:
# from keras.layers import Dropout
# from keras import regularizers, optimizers
# from keras.layers import Input, Conv1D, Dense, Flatten, Activation, UpSampling1D, MaxPooling1D, ZeroPadding1D, TimeDistributed
# from keras.models import Model, load_model
# from keras.layers.core import Reshape
#
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import RepeatVector
# from keras.layers import TimeDistributed
# from keras.utils import plot_model
#
# from keras.layers import Input, LSTM, RepeatVector
# from keras.models import Model

In [2]:
# Load datasets

ptb_char_train = datasets.Dataset("ptb.char.train").data
ptb_char_valid = datasets.Dataset("ptb.char.valid").data
ptb_word_train = datasets.Dataset("ptb.train").data
ptb_word_valid = datasets.Dataset("ptb.valid").data
ptb_word_test = datasets.Dataset("ptb.test").data

wikitext2_train = datasets.Dataset("wiki.train").data
wikitext2_valid = datasets.Dataset("wiki.valid").data
wikitext2_test = datasets.Dataset("wiki.test").data

enwik9_train = datasets.Dataset("enwik.train", os.path.join("C:\\", "Users", "Harry", ".keras", "datasets")).data
enwik9_valid = datasets.Dataset("enwik.valid", os.path.join("C:\\", "Users", "Harry", ".keras", "datasets")).data
enwik9_test = datasets.Dataset("enwik.test", os.path.join("C:\\", "Users", "Harry", ".keras", "datasets")).data


5000000

In [ ]:
class Hyperparameters():
    r = 5
    k = int((90 + 40) / 2) # k < min(m,n)
    hidden_states = 4

class Model(object):
    def __init__(self, params):
        # model parameters
        self.r = params.r
        self.k = params.k
        self.lstm = tf.keras.layers.LSTM(params.hidden_states)

    def mogrify(self, x_init, h_0, q, r):
        x = x_init
        h_prev = h_0
        for i in range(1, self.r+1):
            if i % 2 != 0:
                x = 2 * tf.sigmoid(tf.matmul(q[i-1], h_prev)) * x
                # print("odd %s, %s" % (i, tf.shape(x)))
            else:
                h_prev = 2 * tf.sigmoid(tf.matmul(r[i-1], x)) * h_prev
                # print("even %s, %s" % (i, tf.shape(h_prev)))

        return x, h_prev

    def matrix_decomposition(self, m, n):
        # currently not decomposition
        # k < min(m,n)

        q_left = tf.random.normal([m, self.k])
        q_right = tf.random.normal([self.k, n])

        r_left = tf.random.normal([n, self.k])
        r_right = tf.random.normal([self.k, m])

        return tf.matmul(q_left, q_right), tf.matmul(r_left, r_right)

    def __call__(self, x):
        return

In [ ]:
d = 100
m = d
n = 70
a = 10

x = tf.random.normal([m, a]) # d x a
h_0 = tf.random.normal([n, a]) # n x a

params = Hyperparameters()
model = Model(params)

q = [] # m x n, where m == d
r = [] # n x m, where m == d, it's just transposed Q
for i in range(model.r):
    q_tmp, r_tmp = model.matrix_decomposition(m, n)
    q.append(q_tmp)
    r.append(r_tmp)

model.mogrify(x, h_0, q, r)

In [ ]:
lstm_cell  = tf.keras.layers.LSTM(1, stateful=True)
inputs = tf.random.normal([3, 10, 8])
a = lstm_cell(inputs)
print(a.shape)
states_lstm = lstm_cell.states
states_lstm[0].shape

In [ ]:
class Model(object):
    def __init__(self, output_dim, r=5, k=65):
        # model parameters
        self.r = r
        self.k = k
        self.output_dim = output_dim
        self.lstm = tf.keras.layers.LSTM(self.output_dim, stateful=True, return_state=True)

    def forward(self, x, state = None):

        x, h_prev = self.mogrify(x, )

        res = self.lstm(x, initial_state=None)

        return res

    def mogrify(self, x_init, h_0, q, r):
        x = x_init
        h_prev = h_0
        for i in range(1, self.r+1):
            if i % 2 != 0:
                x = 2 * tf.sigmoid(tf.matmul(q[i-1], h_prev)) * x
                # print("odd %s, %s" % (i, tf.shape(x)))
            else:
                h_prev = 2 * tf.sigmoid(tf.matmul(r[i-1], x)) * h_prev
                # print("even %s, %s" % (i, tf.shape(h_prev)))

        #inputs = tf.random.normal([32, 10, 8])
        #lstm_cell(inputs, initial_state=tf.zeros([3, 3]))

        return x, h_prev

    def matrix_decomposition(self, m, n):
        # currently not decomposition
        # k < min(m,n)

        q_left = tf.random.normal([m, self.k])
        q_right = tf.random.normal([self.k, n])

        r_left = tf.random.normal([n, self.k])
        r_right = tf.random.normal([self.k, m])

        return tf.matmul(q_left, q_right), tf.matmul(r_left, r_right)
    #def __call__(self, x):
    #    return



In [ ]:
inputs = tf.random.normal([32, 10, 8])
outputs = tf.random.normal([32,1])
lstm_model = Model(1)
a = lstm_model.forward(inputs)

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()



In [ ]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(tf.subtract(target_y, predicted_y)))

In [ ]:
@tf.function
def train(model, inputs, outputs, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(outputs, model.forward(inputs))
    grads = t.gradient(current_loss, model.lstm.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.lstm.trainable_variables))

In [ ]:
#train_step(inputs, [0,1,0])
loss(lstm_model.forward(inputs)[0],lstm_model.forward(inputs)[2])

In [ ]:
train(lstm_model, inputs, outputs, 0.001)

In [ ]:
lstm_model.forward(inputs)[0]

In [ ]:
inputs
